In [2]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam, RMSprop

In [3]:

# Set the path to the MRL eyes dataset
dataset_path = "/content/drive/MyDrive/MRLDATA"

# Define image dimensions
img_width, img_height = 64, 64
num_channels = 3  # RGB images have 3 channels


# Load and preprocess the dataset
def load_dataset():
    images = []
    labels = []
    
    for folder_name in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder_name)
        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                if file_name.endswith(".png"):
                    file_path = os.path.join(folder_path, file_name)
                    image = Image.open(file_path).resize((img_width, img_height))
                    # Convert image to RGB if needed
                    if image.mode != "RGB":
                        image = image.convert("RGB")
                    images.append(np.array(image))
                    labels.append(float(folder_name))
    
    images = np.array(images) / 255.0
    labels = np.array(labels)
    
    return images, labels

In [4]:
# Load and split the dataset into training, validation, and testing sets
images, labels = load_dataset()
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)


In [14]:
from sklearn.metrics import accuracy_score, f1_score, precision_score
# Define the CNN model
def create_model(optimizer, num_filters, filter_size):
    model = Sequential()
    model.add(Conv2D(num_filters, (filter_size, filter_size), activation='relu', input_shape=(img_width, img_height, num_channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(num_filters, (filter_size, filter_size), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(num_filters, (filter_size, filter_size), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model with the specified optimizer
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae', 'accuracy'])
    
    return model

In [15]:
# Create KerasRegressor for scikit-learn compatibility
keras_regressor = KerasRegressor(build_fn=create_model, verbose=1)

# Define hyperparameters to tune
param_grid = {
    'epochs': [5, 10],
    'batch_size': [32, 64],
    'optimizer': [Adam(), RMSprop()],
    'num_filters': [32, 64],
    'filter_size': [3, 5]
}

# Perform grid search with cross-validation
grid = GridSearchCV(estimator=keras_regressor, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)
# Get the best model and its hyperparameters
best_model = grid_result.best_estimator_
best_params = grid_result.best_params_
print("Best Hyperparameters:", best_params)

Epoch 1/5


<ipython-input-15-ee7ff3f3005a>:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_regressor = KerasRegressor(build_fn=create_model, verbose=1)


17/17 [==============================] - 1s 5ms/step - loss: 0.1413 - mae: 0.3221 - accuracy: 0.5095
Epoch 2/5
17/17 [==============================] - 0s 4ms/step - loss: 0.0774 - mae: 0.2203 - accuracy: 0.5973
Epoch 3/5
17/17 [==============================] - 0s 4ms/step - loss: 0.0433 - mae: 0.1468 - accuracy: 0.6508
Epoch 4/5
17/17 [==============================] - 0s 4ms/step - loss: 0.0378 - mae: 0.1289 - accuracy: 0.6641
Epoch 5/5
9/9 [==============================] - 0s 4ms/step - loss: 0.0362 - mae: 0.1285 - accuracy: 0.7176
Epoch 1/5
17/17 [==============================] - 1s 5ms/step - loss: 0.1518 - mae: 0.3419 - accuracy: 0.4962
Epoch 2/5
17/17 [==============================] - 0s 5ms/step - loss: 0.1164 - mae: 0.2825 - accuracy: 0.5363
Epoch 3/5
17/17 [==============================] - 0s 4ms/step - loss: 0.0675 - mae: 0.1971 - accuracy: 0.6527
Epoch 4/5
17/17 [==============================] - 0s 4ms/step - loss: 0.0544 - mae: 0.1640 - accuracy: 0.6698
Epoch 5/5
9/9

In [17]:
# Make predictions on the validation set
y_val_pred = best_model.predict(X_val)

# Calculate evaluation metrics for the validation set
val_mae = np.mean(np.abs(y_val_pred - y_val))
print("Validation MAE:", val_mae)

# Make predictions on the test set
y_test_pred = best_model.predict(X_test)

# Calculate evaluation metrics for the test set
test_mae = np.mean(np.abs(y_test_pred - y_test))
print("Test MAE:", test_mae)

6/6 [==============================] - 0s 4ms/step
Validation MAE: 0.08780611365945382
6/6 [==============================] - 0s 4ms/step
Test MAE: 0.07722864222861606


In [13]:
# Load and preprocess the sample image
# /content/drive/MyDrive/testimages/almostclosed.png
# /content/drive/MyDrive/testimages/almostopen.png
# /content/drive/MyDrive/testimages/closedeye.png
# /content/drive/MyDrive/testimages/openeye.png

sample_image = Image.open("/content/drive/MyDrive/testimages/almostclosed.png").resize((img_width, img_height))
sample_image = sample_image.convert("RGB")
sample_image = np.array(sample_image) / 255.0
sample_image = np.expand_dims(sample_image, axis=0)

# Make predictions on the sample image
prediction = best_model.predict(sample_image)
predicted_eye_openness = prediction

print("Predicted Eye Openness:", predicted_eye_openness)


1/1 [==============================] - 0s 20ms/step
Predicted Eye Openness: 0.47070512


In [10]:
sample_image = Image.open("/content/drive/MyDrive/testimages/almostopen.png").resize((img_width, img_height))
sample_image = sample_image.convert("RGB")
sample_image = np.array(sample_image) / 255.0
sample_image = np.expand_dims(sample_image, axis=0)

# Make predictions on the sample image
prediction = best_model.predict(sample_image)
predicted_eye_openness = prediction

print("Predicted Eye Openness:", predicted_eye_openness)


1/1 [==============================] - 0s 22ms/step
Predicted Eye Openness: 0.46508747


In [11]:
sample_image = Image.open("/content/drive/MyDrive/testimages/closedeye.png").resize((img_width, img_height))
sample_image = sample_image.convert("RGB")
sample_image = np.array(sample_image) / 255.0
sample_image = np.expand_dims(sample_image, axis=0)

# Make predictions on the sample image
prediction = best_model.predict(sample_image)
predicted_eye_openness = prediction

print("Predicted Eye Openness:", predicted_eye_openness)

1/1 [==============================] - 0s 22ms/step
Predicted Eye Openness: 0.15814158


In [12]:
sample_image = Image.open("/content/drive/MyDrive/testimages/openeye.png").resize((img_width, img_height))
sample_image = sample_image.convert("RGB")
sample_image = np.array(sample_image) / 255.0
sample_image = np.expand_dims(sample_image, axis=0)

# Make predictions on the sample image
prediction = best_model.predict(sample_image)
predicted_eye_openness = prediction

print("Predicted Eye Openness:", predicted_eye_openness)

1/1 [==============================] - 0s 23ms/step
Predicted Eye Openness: 0.92933303
